In [1]:
import argparse

import numpy as np
import tqdm
from torch import nn, optim
import sys
sys.path.append('/home/anuj/Desktop/Work/TU_Delft/research/implement/learning_to_meta-learn')
sys.path

from src.zoo.delpo_utils import setup, inner_adapt_delpo
from src.utils2 import Profiler
import wandb

In [2]:
profiler = Profiler('DELPO_{}_{}-way_{}-shot_{}-queries'.format('mini_imagenet', 5, 5, 10), 'exp1')

In [3]:
l = [[1,2,3,4,5,6,7], [1,2,3,4,5,6,7]]
profiler.log_csv(l, 'train')

In [4]:
l = [1,2,3,4,5,6,7]
profiler.log_csv(l, 'valid')

In [2]:

# ##############
# # Parameters #
# ##############
# parser = argparse.ArgumentParser()
# parser.add_argument('--dataset', type=str, default='mini_imagenet')
# parser.add_argument('--root', type=str, default='../../data')
# # parser.add_argument('--n-ways', type=int)
# # parser.add_argument('--k-shots', type=int)
# # parser.add_argument('--q-shots', type=int)
# # parser.add_argument('--inner-adapt-steps-train', type=int)
# # parser.add_argument('--inner-adapt-steps-test', type=int)
# # parser.add_argument('--inner-lr', type=float)
# # parser.add_argument('--meta-lr', type=float)
# # parser.add_argument('--meta-batch-size', type=int)
# # parser.add_argument('--iterations', type=int)
# # parser.add_argument('--order', type=str)
# # parser.add_argument('--device', type=str)
# # parser.add_argument('--download', type=str)

# args = parser.parse_args()
# # if args.order == 'True':
# #     args.order = True
# # elif args.order == 'False':
# #     args.order = False

# # if args.download == 'True':
# #     args.download = True
# # elif args.download == 'False':
# #     args.download = False

args = {'dataset': 'miniimagenet', 'root': '../../dataset/mini_imagenet', 'n_ways': 5, 'k_shots': 1, 'q_shots': 1, 'inner_adapt_steps_train': 5, 'inner_adapt_steps_test': 5,
        'inner_lr': 0.001, 'meta_lr': 0.0001, 'meta_batch_size': 1, 'iterations': 3500, 'wt_ce': 1, 'klwt': False, 'rec_wt': 1, 'beta_l': 1, 'beta_s': 1, 'order': False, 'device': 'cpu'}

In [3]:
wandb.init(project="meta", entity='anujinho', config={})
wandb.config.update(args)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: anujinho (use `wandb login --relogin` to force relogin)


In [4]:

# Generating Tasks, initializing learners, loss, meta - optimizer
train_tasks, valid_tasks, test_tasks, learner = setup(
    args['dataset'], args['root'], args['n_ways'], args['k_shots'], args['q_shots'], args['order'], args['inner_lr'], args['device'], download=False)
opt = optim.Adam(learner.parameters(), args['meta_lr'])
reconst_loss = nn.MSELoss(reduction='none')
# if args.order == False:
#     profiler = Profiler('DELPO_{}_{}-way_{}-shot_{}-queries'.format(args.dataset,
#                         args.n_ways, args.k_shots, args.q_shots))
#     prof_test = Profiler('DELPO_test_{}_{}-way_{}-shot_{}-queries'.format(
#         args.dataset, args.n_ways, args.k_shots, args.q_shots))
# elif args.order == True:
#     profiler = Profiler('FO-DELPO_{}_{}-way_{}-shot_{}-queries'.format(
#         args.dataset, args.n_ways, args.k_shots, args.q_shots))
#     prof_test = Profiler('FO-DELPO_test_{}_{}-way_{}-shot_{}-queries'.format(
#         args.dataset, args.n_ways, args.k_shots, args.q_shots))

In [5]:

## Training ##
for iter in tqdm.tqdm(range(args['iterations'])):
    opt.zero_grad()
    
    for batch in range(args['meta_batch_size']):
        ttask = train_tasks.sample()
        model = learner.clone()
        if bool(iter%100==0):
            evaluation_loss, evaluation_accuracy, reconst_img, query_imgs = inner_adapt_delpo(
                ttask, reconst_loss, model, args['n_ways'], args['k_shots'], args['q_shots'], args['inner_adapt_steps_train'], args['device'], True, args)
        else:
            evaluation_loss, evaluation_accuracy = inner_adapt_delpo(
                ttask, reconst_loss, model, args['n_ways'], args['k_shots'], args['q_shots'], args['inner_adapt_steps_train'], args['device'], False, args)
        
        evaluation_loss['elbo'].backward()

        # Logging per train-task losses and accuracies
        wandb.log(dict({f"train/{key}": loss.item() for _, (key, loss) in enumerate(evaluation_loss.items())},
                  **{'train/accuracies': evaluation_accuracy.item(), 'train/task': (iter*args['meta_batch_size'])+batch}))
        
    rimages = wandb.Image(reconst_img, caption="Reconstructed Query Images")  
    qimages = wandb.Image(query_imgs, caption="Query Images")
    wandb.log({"reconst_examples": rimages, "gt_examples": qimages})
        
    vtask = valid_tasks.sample()
    model = learner.clone()
    validation_loss, validation_accuracy = inner_adapt_delpo(
        vtask, reconst_loss, model, args['n_ways'], args['k_shots'], args['q_shots'], args['inner_adapt_steps_train'], args['device'], False, args)

    # Logging per validation-task losses and accuracies
    wandb.log(dict({f"valid/{key}": loss.item() for _, (key, loss) in enumerate(validation_loss.items())},
              **{'valid/accuracies': validation_accuracy.item(), 'valid/task': iter}))

   
    for p in learner.parameters():
        p.grad.data.mul_(1.0 / args['meta_batch_size'])
    opt.step()

#torch.save(learner, f='../repro')

## Testing ##
print('Testing on held out classes')

for i, tetask in enumerate(test_tasks):
    wandb.define_metric("accuracies", summary="max")
    wandb.define_metric("accuracies", summary="mean")

    model = learner.clone()
    #tetask = test_tasks.sample()
    evaluation_loss, evaluation_accuracy = inner_adapt_delpo(
        tetask, reconst_loss, model, args['n_ways'], args['k_shots'], args['q_shots'], args['inner_adapt_steps_test'], args['device'], False, args)
    
    # Logging per test-task losses and accuracies
    wandb.log(dict({f"test/{key}": loss.item() for _, (key, loss) in enumerate(evaluation_loss.items())},
                **{'test/accuracies': evaluation_accuracy.item(), 'test/task': i}))



  0%|          | 0/3500 [00:00<?, ?it/s]/home/anuj/anaconda3/envs/torch/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448278899/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
  6%|▋         | 223/3500 [07:47<1:54:35,  2.10s/it]


KeyboardInterrupt: 

wandb: Network error (ConnectionError), entering retry loop.
wandb: Network error resolved after 0:00:07.945814, resuming normal operation.
